In [0]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import seaborn as sns
#import mlflow
#import mlflow.lightgbm
import seaborn as sns

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective':  'poisson',
        'n_jobs': -1,
        'seed': 15,
        'learning_rate': 0.05, 
        'bagging_fraction': 0.9,
        'bagging_freq': 10, 
        'colsample_bytree': 0.75,
        'num_iterations':  50000,
        'early_stopping_round': 500}

In [0]:
data_folder = '/content/drive/My Drive/Kaggle_M5/00_data/'
submission = pd.read_csv(data_folder + 'sample_submission.csv')

In [0]:
evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation_rows = [row for row in submission['id'] if 'validation' in row] 
validation_csv = submission[submission['id'].isin(validation_rows)]


In [9]:
data_folder = '/content/drive/My Drive/Kaggle_M5/01_preprocessed_data/'

i = 'smooth'
X_val = pd.read_pickle(data_folder +  "X_val_{}.pkl".format(i))
X_val.columns

not_features = ['id', 'd', 'date', 'start_date', 'demand_type', 'demand']
X_val.columns[~X_val.columns.isin(not_features) ]

Index(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'event_name_1', 'event_type_1', 'year', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'days_from_start',
       'start_date_from_start', 'sell_price', 'lag_28', 'lag_29', 'lag_30',
       'lag_31', 'lag_32', 'lag_33', 'lag_34', 'lag_35', 'rmean_28_3',
       'rmean_28_7', 'ADI', 'CV2',
       'demand__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'demand__ar_coefficient__k_10__coeff_3', 'avg_price', 'avg_demand',
       'max_demand', 'std_demand', 'non_zero_seq_len', 'zero_seq_len',
       'avg_weekday_demand', 'quantile025_week', 'max_weekday_demand',
       'cheaper_than_usual'],
      dtype='object')

In [10]:
len(X_val.columns[~X_val.columns.isin(not_features) ])

44

In [0]:
model_folder = '/content/drive/My Drive/Kaggle_M5/05_models/'

validation = pd.DataFrame(columns=submission.columns)
val_score = {'smooth':0, 'erratic':0, 'lumpy':0, 'intermittent':0}
for i in ['intermittent', 'smooth', 'erratic', 'lumpy']:
    print(i)
    X_val = pd.read_pickle(data_folder + "X_val_{}.pkl".format(i))
    X_test = pd.read_pickle(data_folder + "X_test_{}.pkl".format(i))
    X_train = pd.read_pickle(data_folder + "X_train_{}.pkl".format(i))
    y_train = X_train['demand']
    y_val = X_val['demand']
    print(X_train.shape)
    if i == 'intermittent':
        X_train = X_train[X_train['date'] >= '2014-06-01']
        y_train = X_train[X_train['date'] >= '2014-06-01']['demand']
    features = X_train.columns[~X_train.columns.isin(not_features) ]

    train_set = lgb.Dataset(X_train[features],y_train)
    val_set = lgb.Dataset(X_val[features], y_val)
    model = lgb.train(params, train_set, valid_sets = [train_set, val_set], verbose_eval = 100)
    model.save_model(model_folder + 'lgb_' + i + '.txt', num_iteration=model.best_iteration) 


    val_pred = model.predict(X_val[features])
    val_score[i] = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
    print(f'Our val rmse score is {val_score[i]}')

    y_test = model.predict(X_test[features])
    X_test['demand'] = y_test
    predictions = X_test[['id', 'days_from_start', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'days_from_start', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(j + 1) for j in range(28)]
    validation = validation.append(submission[['id']].merge(predictions, on = 'id'))

    

intermittent
(32061790, 50)


In [0]:
val_score

In [0]:
validation = validation.set_index('id')
validation = validation.reindex(index=validation_csv['id'])
validation = validation.reset_index()

In [0]:
validation

In [0]:
final = pd.concat([validation, evaluation])

In [0]:
data_folder = '/content/drive/My Drive/Kaggle_M5/04_submissions/'
final.to_csv(data_folder + 'submission-by_demand_type_poisson.csv', index = False)